In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import pipeline

model_name = "tsmatz/xlm-roberta-ner-japanese"
classifier = pipeline("token-classification", model=model_name, device="cuda:0")
result = classifier("鈴木は4月の陽気の良い日に、鈴をつけて熊本県の阿蘇山に登った")
print(result)

[{'entity': 'PER', 'score': 0.9993142, 'index': 1, 'word': '▁', 'start': 0, 'end': 1}, {'entity': 'PER', 'score': 0.9993973, 'index': 2, 'word': '鈴', 'start': 0, 'end': 1}, {'entity': 'PER', 'score': 0.9992887, 'index': 3, 'word': '木', 'start': 1, 'end': 2}, {'entity': 'LOC', 'score': 0.9991264, 'index': 14, 'word': '熊本', 'start': 19, 'end': 21}, {'entity': 'LOC', 'score': 0.9980952, 'index': 15, 'word': '県', 'start': 21, 'end': 22}, {'entity': 'LOC', 'score': 0.99906176, 'index': 17, 'word': '阿', 'start': 23, 'end': 24}, {'entity': 'LOC', 'score': 0.9990877, 'index': 18, 'word': '蘇', 'start': 24, 'end': 25}, {'entity': 'LOC', 'score': 0.9983518, 'index': 19, 'word': '山', 'start': 25, 'end': 26}]


In [3]:
from utils import flatten, get_consecutive_name_entities, partition_words, find_first_non_consecutive_substring
from utils import fix_repeated_chars, find_example_sentences, has_kana, load_config, concat_kanji_rubi, find_rubi, toggle_kana
from utils import parse_gpt_json, remove_common_suffix, extract_ruby_from_epub
from tqdm import tqdm 
import fastapi_poe as fp
from epubparser import main
import re
import os

In [4]:
api_key = 'gp13DNOpQh8lBHin55kD6_0js__F7n6WfWQOHarr8P0'

# Create an asynchronous function to encapsulate the async for loop
async def get_responses(api_key, messages):
    final_message = ""
    async for partial in fp.get_bot_response(messages=messages, bot_name="GPT-4", api_key=api_key):
        final_message += partial.text
        print(partial.text, end="")
    return final_message

In [5]:
config = load_config()
epub_files = [os.path.join('output', config['CN_TITLE'], 'input.epub')] 
# epub_files = glob.glob(os.path.join('books/', '**/*.epub'), recursive=True)

book = []
for file in epub_files:
    book += main(file)
book = [concat_kanji_rubi(x) for x in book]

c:\Users\kevin\miniconda3\envs\epub\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
c:\Users\kevin\Github\epubtranslator\epubparser.py:19: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(content, "html5lib")
  0%|          | 0/612 [00:00<?, ?it/s]c:\Users\kevin\Github\epubtranslator\epubparser.py:40: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe i

In [6]:
results = []
for jp_text in tqdm(book):
    results.append(classifier(jp_text))

  0%|          | 7/3704 [00:00<01:59, 30.90it/s]c:\Users\kevin\miniconda3\envs\epub\Lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 3704/3704 [01:49<00:00, 33.86it/s]


In [7]:
entities = {}
ignorable_postfix = ['家の', '様', 'の', 'くん', '家', 'さん', 'と', 'ちゃん', 'っち', '姉', "王女", "先輩", "領", '殿',
                     '連合', '軍', '部隊', '大陸', 'たん', 'が', 'も', 'は', '君', 'ッ']
ignorable_mid = ["王国", "聖国", "帝国", "獣国", "公国", "皇国", '国']
exclusion = ['わたく', 'わたくし', 'きょー', 'きみ', 'アレ', 'テメェ', 
             'わらわ', 'ぼかぁ', 'まんこ', 'カリ', 'おちんぽ', 'メイド', 'オーク', 
             'さん', 'コイツ', 'クリトリス', '日本', 'オーノー', 'アン', 'カミサマ']

### Counting entities, ignoing entities with ignorable postfix and mid
for result in results: 
    for entity in get_consecutive_name_entities(result, score_threshold=0.2):
        if '▁' in entity:
            entity = re.sub(r'▁.*', '', entity)
        if entity in exclusion:
            continue
        flag = False
        for postfix in ignorable_postfix:
            entity_wo = entity[:-len(postfix)]
            if entity.endswith(postfix) and entity_wo in entities:
                if entity in entities:
                    entities[entity_wo] += entities[entity]
                    del entities[entity]
                entities[entity_wo] += 1
                flag = True
                break
        for mid in ignorable_mid:
            if mid in entity:
                entity_wo = entity.split(mid)[0]
                if entity_wo in entities:
                    if entity in entities:
                        entities[entity_wo] += entities[entity]
                        del entities[entity]
                    entities[entity_wo] += 1
                    flag = True
                break
        if flag:
            continue
        if "の" in entity:
            continue
        if entity in entities:
            entities[entity] += 1
        else:
            entities[entity] = 1

In [8]:
entities.keys()

dict_keys(['レア', 'すえた匂い', 'ケリー', '文化的なコミュニケーション', 'エラーメッセージ', '考察', '使役', '元山猫盗賊団アジト予定地', 'お出かけ', '狩り', '氷狼', 'アリと狼', 'かしこさ', '情報が渋滞してる', '元女王国予定地跡地', 'ウェイン、草原に立つ', 'ブラン・ニュー・ゲーム', '蜾蠃娘子', 'インベントリ', '眷属強化', 'ストラテジー', '心に潜む闇', 'プレイヤーキラー', 'ボーナスステージ', 'システムメッセージ', '開始', '座標認識', '墓', '好', '地。ただし廃墟', '卵とリビング', '第一回大規模イベント開始', '鎧坂さんからは逃げられない', 'ラスボス二段変身ごっこ', '変態だけを殺す機械', '水洗', 'ギル、って呼んでくれ', 'エキシビション', '少し面倒く', '擬態', 'カンファートレント', 'ひさしぶり。ちょっと痩せた?」', 'クエストを受注しました。なお', '世界樹とハイ・エルフ', '選択肢', '魔王誕生', '臨時国会', 'もふ', '侵攻計画', '蹂躙', '深刻な辺境破壊問題', 'ボーナスステージ再び', 'ブ', 'デイウォーカー', 'ブラン', 'ウェイン', 'ウェイン、王都に立つ', 'マルチプル・プレイ', 'リザルト', '魔王覚醒', 'リベンジ', 'ダグラス・オコーネル', 'NPCは侮れない', '今日は廃墟でマラソン大会', '邂逅', 'ヨロイ・ザ・カサン', '公式SNS', '置き土産', 'コネート', '再会', 'アルフ・ライラ・ワ・ライラ', 'ライラ', 'じゃあいい', 'ばか', '姉妹', 'と思ったらコレだよ', 'オーラル革命', 'ダンジョン・コンバージョン', '巣立ち', '新しい朝がきた', 'ネコババ', '説明好きなとことか', '外部顧問', 'すみませんフレに呼ばれますね^^', '火山遠征', 'エルダーロックゴーレム', '神託', '邪道をゆく者', '選考対象外', '限定転移サービス実装', 'ダンログ評価☆3', 'エルフ', '3秒ルール', '早速悪用される新サービス', 'なかまをよんだ!」', 'しかしだれも

In [12]:
# full_text = '\n'.join(book)
# # rubi = find_rubi(entities, full_text.replace('(', '（').replace(')', '）'))
# rubi = find_rubi(entities, full_text)
rubi = extract_ruby_from_epub(epub_files[0])
rubi

{'す': '空',
 'ライフポイント': 'ＬＰ',
 'や': '飲',
 'イコール': '＝',
 'マナポイント': 'ＭＰ',
 '・': '妹',
 'はくま': '白魔',
 'ぎんか': '銀花',
 'なめ': '鞣',
 '再ログイン': 'リログ',
 'グォレンダァ': '５連打',
 'さかい': '境',
 'スガル': '蜾蠃',
 'かぶり': '頭',
 'プレイヤーキラー': 'ＰＫ',
 'ひとけ': '人気',
 'スクワイア・ゾンビ': '吸血鬼の従者',
 'レヴナント': '意志ある死者',
 'レッサーヴァンパイア': '下級吸血鬼',
 'もてな': '饗',
 '眠っ': 'ログアウトし',
 'やじり': '鏃',
 'かたびら': '帷子',
 'ふがし': '麩菓子',
 'すさ': '退',
 'いころ': '射殺',
 'ストイック': '禁欲的',
 'つつもたせ': '美人局',
 'もや': '靄',
 '生命力': 'ＬＰ',
 'スパルトイ': '蒔かれた者',
 'ま': '蒔',
 'ぜんしん': '漸進',
 'おもんぱか': '慮',
 '経験値': '命',
 'ログアウトして': '寝て',
 'しつら': '設',
 'むくろ': '骸',
 '我が家': 'リーベ大森林',
 'かえ': '反',
 'かげ': '翳',
 'にぶ': '鈍',
 'よ': '撚',
 'にかわ': '膠',
 'みなぎ': '漲',
 'トゥルーヴァンパイア': '真祖吸血鬼',
 'たまわ': '賜',
 'ふかん': '俯瞰',
 'はびこ': '蔓延',
 'センチュリオン': '百卒長',
 'すうせい': '趨勢',
 'ね': '睨',
 'ミドル・ヒール': '中回復',
 'リジェネレーション': '再生',
 'エリア・リトルヒール': '範囲小回復',
 'すく': '掬',
 'しゃく': '癪',
 'クリティカル': 'ヘッドショット',
 'あった': '暖',
 'アーティファクト': '秘遺物',
 'はま': '嵌',
 'かせ': '枷',
 'まみ': '見',
 'みまか': '身罷',
 'さが': '性',
 'イモータルルーラー': '不死者の王

In [13]:
rubi = {k: v for k, v in rubi.items() if not has_kana(v) and has_kana(k)}
rubi

{'す': '空',
 'ライフポイント': 'ＬＰ',
 'や': '飲',
 'イコール': '＝',
 'マナポイント': 'ＭＰ',
 'はくま': '白魔',
 'ぎんか': '銀花',
 'なめ': '鞣',
 'グォレンダァ': '５連打',
 'さかい': '境',
 'スガル': '蜾蠃',
 'かぶり': '頭',
 'プレイヤーキラー': 'ＰＫ',
 'ひとけ': '人気',
 'レッサーヴァンパイア': '下級吸血鬼',
 'もてな': '饗',
 'やじり': '鏃',
 'かたびら': '帷子',
 'ふがし': '麩菓子',
 'すさ': '退',
 'いころ': '射殺',
 'ストイック': '禁欲的',
 'つつもたせ': '美人局',
 'もや': '靄',
 'ま': '蒔',
 'ぜんしん': '漸進',
 'おもんぱか': '慮',
 'しつら': '設',
 'むくろ': '骸',
 'かえ': '反',
 'かげ': '翳',
 'にぶ': '鈍',
 'よ': '撚',
 'にかわ': '膠',
 'みなぎ': '漲',
 'トゥルーヴァンパイア': '真祖吸血鬼',
 'たまわ': '賜',
 'ふかん': '俯瞰',
 'はびこ': '蔓延',
 'センチュリオン': '百卒長',
 'すうせい': '趨勢',
 'ね': '睨',
 'ミドル・ヒール': '中回復',
 'リジェネレーション': '再生',
 'エリア・リトルヒール': '範囲小回復',
 'すく': '掬',
 'しゃく': '癪',
 'あった': '暖',
 'アーティファクト': '秘遺物',
 'はま': '嵌',
 'かせ': '枷',
 'まみ': '見',
 'みまか': '身罷',
 'さが': '性',
 'しい': '弑',
 'しらみつぶ': '虱潰',
 'か': '彼',
 'これ': '『隠伏』',
 'ライラ': '■■■',
 'レアちゃん': '■■■',
 'グレーター': '上級',
 'ヴァンパイア': '吸血鬼',
 'アバター': '人',
 'てい': '体',
 'スカウト': '斥候',
 'こな': '熟',
 'やぶさ': '吝',
 'バロネス': '男爵',
 'なに': '何',


In [11]:
multi_rubi = {re.sub(r'\s', '', k): v for k, v in rubi.items() if len(v) > 1}
print(multi_rubi)

{}


In [ ]:
response = None
if not len(multi_rubi) == 0:
    with open('resource/rubi_prompt.txt', 'r') as f:
        ruby_prompt = f.read()
        
    prompt = ruby_prompt + str(multi_rubi)
    message = fp.ProtocolMessage(role="user", content=prompt)
    response = await get_responses(api_key, [message])

In [ ]:
if not len(multi_rubi) == 0 and 'response' in locals() and response is not None:
    dictionary = parse_gpt_json(response)
    dictionary.update(rubi)
    print(dictionary)
else:
    dictionary = rubi

In [ ]:
# Dump the final dictionary to CN_TITLE/names.json
import json
with open(os.path.join('output', config['CN_TITLE'], 'names.json'), 'w') as f:
    json.dump(dictionary, f, ensure_ascii=False, indent=4)

In [ ]:
def filter(entity, count):
    for name in dictionary.keys():
        if entity in name or name in entity:
            return False
    for name in dictionary.values():
        if entity in name or name in entity:
            return False
    return len(entity) > 1 and count > 1 and fix_repeated_chars(entity) == entity and has_kana(entity) 

entities = {entity: count for entity, count in entities.items() if filter(entity, count)}
entities = dict(sorted(entities.items(), key=lambda item: item[1], reverse=True))
names = partition_words(entities.keys(), 30)

names

In [ ]:
example_sentences = find_example_sentences(flatten([list(s) for s in names]), book)
responses = []

with open('resource/namedetect_prompt_2.txt', 'r') as f:
    prompt_2 = f.read()

for name in names:
    name = sorted(list(name))
    prompt = ''
    for n in name:
        if n in example_sentences:
            sentence = example_sentences[n]
            if len(sentence) > 100:
                idx = sentence.find(n)
                sentence = sentence[idx - 50: idx + 50]
            prompt += n + '：' + sentence + '\n'
    prompt = prompt_2 + prompt + "\n\n----- 现在请回答日文名词，用中文解释"
    
    print(prompt)
    print('\n' + '#' * 80 + '\n')
    
    message = fp.ProtocolMessage(role="user", content=prompt)
    responses.append(await get_responses(api_key, [message]))
    print('\n' + '#' * 80 + '\n')
    

In [ ]:
name_set = set().union(*names)
filtered_names = []
for response in responses:
    new_names = set()
    for line in response.split('\n'):
        line = line.strip()
        if "解释" in line:
            break
        if "：" in line or len(line.strip()) == 0:
            continue
        if line.startswith('- '):
            line = line[2:]
        word = find_first_non_consecutive_substring(line, name_set)
        if word is not None:
            new_names.add(word)
        else:
            new_names.add(line)
    if len(new_names) == 0:
        raise
    filtered_names += new_names

for i, (entity, count) in enumerate(entities.items()):
    if count > 20 and i < 10:  # Top 20 and count > 10
        for name in filtered_names:  # Not extending found names
            if name in entity:
                break
        else:
            filtered_names.append(entity)
            
filtered_names = partition_words(filtered_names, 30)
print(len(filtered_names))
filtered_names

In [ ]:
with open('resource/namedetect_prompt_1.txt', 'r') as f:
    prompt_1 = f.read()
    
responses_ = []
for name in filtered_names:
    name = sorted(list(name))
    prompt = ''
    for n in name:
        if n in example_sentences:
            prompt += n + '：' + example_sentences[n] + '\n'
    prompt = prompt_1 + prompt
    print(prompt)
    message = fp.ProtocolMessage(role="user", content=prompt)
    responses_.append(await get_responses(api_key, [message]))
    print()

In [ ]:
final_dictionary = {}
for response in responses_:
    final_dictionary.update(parse_gpt_json(response))
final_dictionary.update(dictionary)
to_delete = []
to_add = {}
for key, val in final_dictionary.items():
    if val.endswith('君') and key.endswith('さん'):
        to_delete.append(key)
        to_add[key[:-1]] = val[:-1]
    key_new, val_new = remove_common_suffix(key, val)
    if key_new != key:
        to_delete.append(key)
        to_add[key_new] = val_new
        
for key in to_delete:
    del final_dictionary[key]
for key, val in to_add.items():
    final_dictionary[key] = val

print(final_dictionary)

In [ ]:
for keys in final_dictionary.keys():
    if toggle_kana(keys) in final_dictionary:
        if final_dictionary[keys] != final_dictionary[toggle_kana(keys)]:
            print(keys, final_dictionary[keys], final_dictionary[toggle_kana(keys)])
            if keys in dictionary:
                final_dictionary[keys] = dictionary[keys]
                final_dictionary[toggle_kana(keys)] = dictionary[keys]
            elif toggle_kana(keys) in dictionary:
                final_dictionary[keys] = dictionary[toggle_kana(keys)]
                final_dictionary[toggle_kana(keys)] = dictionary[toggle_kana(keys)]
            else:
                final_dictionary[keys] = final_dictionary[toggle_kana(keys)]
final_dictionary[config["JP_TITLE"]] = config["CN_TITLE"]

In [ ]:
# Dump the final dictionary to CN_TITLE/names.json
import json
with open(os.path.join('output', config['CN_TITLE'], 'names.json'), 'w') as f:
    json.dump(final_dictionary, f, ensure_ascii=False, indent=4)